# Merging and uploading the fine tuned model to Hugging Face

## Step 1: Installing and importing the libraries

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import PeftModel

## Step 2: Loading the tokenizer, base model and adaptor model

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
adaptor_model = "Binura/BMP-MineEngInsight"

In [ ]:
mistral_tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code = True)
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
mistral_tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
mistral_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype = torch.bfloat16,
        return_dict = True,
        low_cpu_mem_usage = True,
        device_map={"": 0},
        trust_remote_code = True,
)

fine_tuned_model = PeftModel.from_pretrained(mistral_model_reload, adaptor_model)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

## Step 3: Merging base model and adaptor model

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(mistral_model_reload, adaptor_model)
fine_tuned_model = fine_tuned_model.merge_and_unload()

## Step 4: Uploading the fine tuned model to Hugging Face

In [ ]:
directory_name = "BMP-MineEngInsight"

if not os.path.exists(directory_name):
    os.makedirs(directory_name)

In [ ]:
fine_tuned_model.push_to_hub(adaptor_model, use_temp_dir=False)
mistral_tokenizer.push_to_hub(adaptor_model, use_temp_dir=False)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Binura/BMP-MineEngInsight/commit/86eaeccb1e380e1cba42085ea1e8b244726d9bf2', commit_message='Upload tokenizer', commit_description='', oid='86eaeccb1e380e1cba42085ea1e8b244726d9bf2', pr_url=None, pr_revision=None, pr_num=None)